<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importar-Librerías" data-toc-modified-id="Importar-Librerías-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importar Librerías</a></span></li><li><span><a href="#Preprocesamiento-de-Datos" data-toc-modified-id="Preprocesamiento-de-Datos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocesamiento de Datos</a></span><ul class="toc-item"><li><span><a href="#Construcción-del-Dataset-de-Productos" data-toc-modified-id="Construcción-del-Dataset-de-Productos-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Construcción del Dataset de Productos</a></span><ul class="toc-item"><li><span><a href="#Lectura-de-los-Datos" data-toc-modified-id="Lectura-de-los-Datos-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Lectura de los Datos</a></span></li><li><span><a href="#Transformación-de-los-Datos" data-toc-modified-id="Transformación-de-los-Datos-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Transformación de los Datos</a></span></li></ul></li></ul></li></ul></div>

# Pulling Data - Mercado Libre API

El propósito de este notebook es realizar el primer paso del reto de DS y ML propuesto por el equipo Meli: Obtener los datos de ciertos productos a través de la [API](https://api.mercadolibre.com/) de Mercado Libre.

En este notebook se establece la conexión con la API y se obtiene através del buscador de Mercado Libre ciertos productos para construir un dataset que será usuado tanto para la parte exploratoria como la del modelo de ML.

## Importar Librerías

In [1]:
# Cambia el ancho de las celdas
from IPython.display import HTML, display
display(HTML(data="""<style>div#notebook-container{width: 60%;}div#menubar-container{width: 65%;} div#maintoolbar-container{width: 99%;}</style>"""))

In [1]:
# Procesamiento de datos
import os
import re
import pandas as pd
import numpy as np
import datetime as dt
import itertools as it

# Visualización
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib.ticker import PercentFormatter
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

## Preprocesamiento de Datos

En esta sección se realiza un proceso general de limpieza y organización de los datos previo a la exploración de los datos.

### Construcción del Dataset de Productos

#### Lectura de los Datos
 
En esta subsección se leen el dataset de productos, se imprimen sus dimensiones, sus columnas y se mencionan sus tipos de variables.

In [4]:
# Se lee el .csv con el datset construido
products = pd.read_csv('../Data/Datasets/PRODUCTOS.csv')
products.columns = products.columns.str.upper() # Pone los nombres de las columnas en mayusculas
print('LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: {}'.format(products.shape))

LAS DIMENSIONES DE LA BASE DE PRODUCTOS SON: (14531, 47)


In [6]:
# Printea la columnas del dataset
products.columns

Index(['ITEM_ID', 'ITEM_SITE_ID', 'ITEM_TITLE', 'ITEM_PRICE',
       'ITEM_SALE_PRICE', 'ITEM_CURRENCY_ID', 'ITEM_AVAILABLE_QUANTITY',
       'ITEM_SOLD_QUANTITY', 'ITEM_BUYING_MODE', 'ITEM_LISTING_TYPE_ID',
       'ITEM_STOP_TIME', 'ITEM_CONDITION', 'ITEM_PERMALINK', 'ITEM_THUMBNAIL',
       'ITEM_THUMBNAIL_ID', 'ITEM_ACCEPTS_MERCADOPAGO', 'ITEM_ORIGINAL_PRICE',
       'ITEM_CATEGORY_ID', 'ITEM_OFFICIAL_STORE_ID', 'ITEM_DOMAIN_ID',
       'ITEM_CATALOG_PRODUCT_ID', 'ITEM_ORDER_BACKEND',
       'ITEM_USE_THUMBNAIL_ID', 'SELLER_ID', 'SELLER_REP_TRANSACTIONS_TOTAL',
       'SELLER_REP_TRANSACTIONS_CANCELED', 'SELLER_REP_RATING_NEG',
       'SELLER_REP_RATING_POS', 'SELLER_REP_RATING_NEU',
       'SELLER_TRANSACTIONS_COMPLETED', 'SELLER_STATUS',
       'SELLER_METRICS_CLAIMS_RATE', 'SELLER_METRICS_CLAIMS_VALUE',
       'SELLER_METRICS_CLAIMS_PERIOD', 'SELLER_METRICS_DELAY_RATE',
       'SELLER_METRICS_DELAY_VALUE', 'SELLER_METRICS_DELAY_PERIOD',
       'SELLER_METRICS_SALES_PERIOD', 'SELL

Se puede ver claramente que se encuentran los siguientes tipos de variables:
    
1. Tipo `ITEM`: Variables asociadas al ítem.


2. Tipo `SELLER`: Variables asociadas al vendedor.


3. Tipo `SHIPPING`: Variables asociadas al envío.


4. Tipo `ADRESS`: Variable que indica el departamento donde se encuentra el ítem.

#### Transformación de los Datos

En esta subsección se arreglan, limpian y transforman el dataset de productos.

In [58]:
# Arreglo a la ADRESS_STATE_ID: para algunos productos el ADRESS_STATE_ID hay que arreglar su dato
# Se uso el ITEM_PERMALINK para entender cual era el verdadero dato de ADRESS_STATE_ID
cambios_de_direccion = {
                        'TUNPUEJPR1gxMDljZA': 'CO-DC', 
                        'TUNPUEFOVGFiZWI3': 'CO-ANT ',
                        'TUNPUENVTmE3NmQ4': 'CO-CUN',
                        'TUNPUFZBTGExNmNjNg': 'CO-VAC',
                        'TUNPUFJJU2ExMWIyYg': 'CO-RIS',
                        'TUNPUEFUTG9mNDk5': 'CO-ATL',
                        'TUNPUFRPTGExNGZkNA': 'CO-TOL',
                        'TUNPUFNBTnJlMjMw': 'CO-SAN',
                        'TUNPUEJPTHI1Mzlk': 'CO-BOL',
                        'TUNPUE1BR2FiZjQ0': 'CO-MAG',
                        'TUNPUENBTHNjODY4': 'CO-CAL',
                        'TUNPUE1FVGExNzFjNQ': 'CO-MET',
                        'TUNPUFFVSW9kYmZm': 'CO-QUI',
                        'TUNPUE5PUnIxNDkyZg': 'CO-NSA', 
                        'TUNPUEFSQ2E4Zjc3': 'CO-SAP',
                        'TUNPUEJPWWE4YzMz': 'CO-BOY',
                        'TUNPUENBVWExM2Q1NQ': 'CO-CAU',
                        'TUNPUENPUmFkZGIw': 'CO-COR',
                        'TUNPUEdVQWExOTYx': 'CO-LAG',
                        'TUNPUE5BUm8xYzk4': 'CO-NAR', 
                        'TUNPUFNVQ2U4ZWQ0': 'CO-SUC',
                        'TUNPUENBU2U2OWIy': 'CO-CAS',
                       }

# Arregla un error que se detecto en las direcciones
products.loc[products['ADRESS_STATE_ID'].isin(cambios_de_direccion.keys()), 'ADRESS_STATE_ID'] = \
products.loc[products['ADRESS_STATE_ID'].isin(cambios_de_direccion.keys()), 'ADRESS_STATE_ID'].map(cambios_de_direccion)

In [120]:
# products.loc[products['ITEM_CURRENCY_ID'] == 'USD', 'ITEM_PRICE']
products['SELLER_REP_RATING_NEU'].isna().sum()

0

In [90]:
## Ajustes al precio 

# Se quitan productos que no tienen precio (generalmente precio a convenir)
# Es decir este analisis aplica para productos con precio fijo
products = products.loc[~products['ITEM_PRICE'].isna()]
products.reset_index(drop=True, inplace=True)

# Se encontro en google la tasa de cambio de dolar a cop el dia 7 de marzo (cuando se descargaron los datos)
exchange_rate_usd_to_cop = 3142.99 
products.loc[products['ITEM_CURRENCY_ID'] == 'USD', 'ITEM_PRICE'] *= exchange_rate_usd_to_cop
products.loc[products['ITEM_CURRENCY_ID'] == 'USD', 'ITEM_CURRENCY_ID'] = 'COP'

# Cambia ITEM_ORIGINAL_PRICE por el ITEM_PRICE como se indica en el documento
products.loc[products['ITEM_ORIGINAL_PRICE'].isna(), 'ITEM_ORIGINAL_PRICE'] = \
products.loc[products['ITEM_ORIGINAL_PRICE'].isna(), 'ITEM_PRICE']

# Crea las variables de descuento absoluto y la tasa
products['ITEM_DISCOUNT_VALUE'] = products['ITEM_ORIGINAL_PRICE'] - products['ITEM_PRICE']
products['ITEM_DISCOUNT_RATE'] = products['ITEM_DISCOUNT_VALUE']/products['ITEM_ORIGINAL_PRICE']

# Crea la variable de venta total: This is an approximation
products['ITEM_SELL_TOTAL'] = products['ITEM_PRICE']*products['ITEM_SOLD_QUANTITY']

In [ ]:
## Ajuste a la condicion del item

# Se cambia nan por not_specified
products.loc[products['ITEM_CONDITION'].isna(), 'ITEM_CONDITION'] = 'not_specified'

In [91]:
for c in products.columns:
    print(c, products[c].nunique())

ITEM_ID 14214
ITEM_SITE_ID 1
ITEM_TITLE 13786
ITEM_PRICE 3474
ITEM_SALE_PRICE 0
ITEM_CURRENCY_ID 2
ITEM_AVAILABLE_QUANTITY 41
ITEM_SOLD_QUANTITY 31
ITEM_BUYING_MODE 2
ITEM_LISTING_TYPE_ID 4
ITEM_STOP_TIME 5504
ITEM_CONDITION 3
ITEM_PERMALINK 14214
ITEM_THUMBNAIL 14143
ITEM_THUMBNAIL_ID 13800
ITEM_ACCEPTS_MERCADOPAGO 2
ITEM_ORIGINAL_PRICE 458
ITEM_CATEGORY_ID 1437
ITEM_OFFICIAL_STORE_ID 203
ITEM_DOMAIN_ID 1098
ITEM_CATALOG_PRODUCT_ID 1127
ITEM_ORDER_BACKEND 50
ITEM_USE_THUMBNAIL_ID 2
SELLER_ID 3766
SELLER_REP_TRANSACTIONS_TOTAL 2285
SELLER_REP_TRANSACTIONS_CANCELED 562
SELLER_REP_RATING_NEG 26
SELLER_REP_RATING_POS 33
SELLER_REP_RATING_NEU 24
SELLER_TRANSACTIONS_COMPLETED 2258
SELLER_STATUS 3
SELLER_METRICS_CLAIMS_RATE 383
SELLER_METRICS_CLAIMS_VALUE 107
SELLER_METRICS_CLAIMS_PERIOD 4
SELLER_METRICS_DELAY_RATE 901
SELLER_METRICS_DELAY_VALUE 162
SELLER_METRICS_DELAY_PERIOD 4
SELLER_METRICS_SALES_PERIOD 4
SELLER_METRICS_SALES_COMPLETED 1245
SELLER_METRICS_CANCELLATIONS_PERIOD 4
SELLER_MET

# Fin del Documento